# Download Open Food Proudcts Data

In [ ]:
!wget https://static.openfoodfacts.org/data/en.openfoodfacts.org.products.csv.gz 
!ls 

#### Import Libraries

In [ ]:
import os
import duckdb
import pandas as pd
import numpy as np
from PIL import Image
import seaborn as sns
from matplotlib import pyplot as plt
from io import BytesIO

# Load Data into DuckDB

In [ ]:
if not os.path.exists("openfoodfacts.duckdb"):

    con = duckdb.connect("openfoodfacts.duckdb")
    
    # Create a table from the GZIP CSV
    con.execute("""
        CREATE OR REPLACE TABLE products AS
        SELECT *
        FROM read_csv_auto(
            'en.openfoodfacts.org.products.csv.gz',
            delim='\t',
            header=True
        );
        """)

# Check **products** table, Number of Columns and few column names

In [ ]:
# list tables
con = duckdb.connect("openfoodfacts.duckdb")
tables = con.execute("PRAGMA show_tables;").fetchdf()
print(tables)

# get all column names
columns = con.execute("PRAGMA table_info('products');").fetchdf()
print("Total Columns:", len(columns))
columns[:10]

In [ ]:
for col in columns['name']:
    print(col)

# View sample products data

In [ ]:
sample_food_products_df = con.execute("SELECT * from products LIMIT 5").df()
sample_food_products_df.head()

In [ ]:
for key, value in sample_food_products_df.loc[1].items():
    if not pd.isna(value):
        print(key, "=", value)

## Completeness Score

In [ ]:
completeness_info = con.execute("SELECT completeness from products").df()
completeness_info.head()

In [ ]:
plt.figure(figsize=(8, 4))
sns.histplot(completeness_info["completeness"].dropna(), bins=30, kde=True)
plt.title("Distribution of Product Completeness Scores")
plt.xlabel("Completeness (0 = low info, 1 = full info)")
plt.ylabel("Number of Products")
plt.show()

* There is peak between 0.3 and 0.5, almost half of the products from overall products are 30% to 50% complete with information.
* Less than 200k products are available with 80% complete information.

## Get all Nutritional Facts Columns

In [ ]:
nutri_cols = [c for c in columns['name'] if '_100g' in c or 'nutrition' in c]
print("Total Nutritional Columns:", len(nutri_cols))
nutri_cols[:10]

In [ ]:
# Select only product info + nutrition columns
con.execute(f"""CREATE OR REPLACE VIEW nutrition
               AS SELECT code, product_name, quantity, product_quantity, brands_en, countries_en, categories_en, serving_size, serving_quantity,
               nutriscore_score, nutriscore_grade, food_groups_en, nutrient_levels_tags, image_url
              {','.join(f'"{col}"' for col in nutri_cols)} FROM products WHERE energy_100g IS NOT NULL
              """)

In [ ]:
# total number of rows
import humanize
c = con.execute("SELECT count(*) as count from products").fetchdf()
print("Total Number of Rows in Products tables:", humanize.intword(c.loc[0, 'count']))


c = con.execute("SELECT count(*) as count from nutrition").fetchdf()
print("Total Number of Rows in Nutrition:", humanize.intword(c.loc[0, 'count']))

# View Sample Data of Nutritional Food Products

In [ ]:
sample_df = con.execute("SELECT * from nutrition LIMIT 5").df()
sample_df.head()

In [ ]:
for key, value in sample_df.loc[3].items():
    if not pd.isna(value):
        print(key, "=", value)

* It may be possible that some food products may not have image associated that shows nutritional info, we can ignore such products for modeling, as images are required in Task 2.
  

## Get all nutritional food prodcuts for which image_nutrition_url is present

In [ ]:
nutrition_df = con.execute("SELECT * from nutrition where image_nutrition_url is not NULL and nutriscore_score is not NULL").df()
print(nutrition_df.shape)
nutrition_df.head()

## Investigate total number product categories and its distribution

In [ ]:
from collections import defaultdict

unique_product_categories = defaultdict(int)

for categories in nutrition_df["categories_en"]:
    if pd.isna(categories):
        unique_product_categories["no_product_category"] += 1
    else:
        for c in categories.split(","):
            unique_product_categories[c] += 1

print("Total Product Categories:", len(unique_product_categories))

In [ ]:
product_category_df = pd.DataFrame({'product_category': unique_product_categories.keys(), 
                                    'frequency': unique_product_categories.values()})

product_category_df

In [ ]:
# check data distribution
product_category_df["frequency"].describe(percentiles=[0.5, 0.9, 0.99])

* Mean = 96.2 average number of products per category
* Std = 2353.37 extremely high — distribution heavy right tail
* 50% (Median) percentile = half the categories have only 1 product
* 90% percentile = 90% of categories have ≤ 15 products
* 99% percentile = only top 1% categories exceed 872 products

## Top N - bar plot

In [ ]:
top_n = 30
df_sorted = product_category_df.sort_values(by="frequency", ascending=False)

top_df = df_sorted.head(top_n)

plt.figure(figsize=(10, 8))
sns.barplot(data=top_df, y="product_category", x="frequency", palette="viridis")
plt.title(f"Top {top_n} Product Categories by Frequency")
plt.xlabel("Number of Products")
plt.ylabel("Category")
plt.show()


* Over 500k food products belong to Plant-based foods and beverages category

# Cumulative Food product distribution by Product category

In [ ]:
df_sorted = product_category_df.sort_values("frequency", ascending=False)
df_sorted["cum_percent"] = df_sorted["frequency"].cumsum() / df_sorted["frequency"].sum() * 100

plt.figure(figsize=(8, 5))
plt.plot(df_sorted["cum_percent"], color="blue")
plt.axhline(80, color="red", linestyle="--", label="80% coverage")
plt.title("Cumulative Product Coverage by Category")
plt.xlabel("Number of Categories (sorted by frequency)")
plt.ylabel("Cumulative % of Products")
plt.legend()
plt.show()

# how many categories cover 80% of products
n_80 = (df_sorted["cum_percent"] <= 80).sum()
print(f"→ {n_80:,} categories (~{n_80 / len(df_sorted):.2%}) cover 80% of all products.")


In [ ]:
product_category_df.to_csv("product_categories.csv", index=False)

# Check Sample data for Particular Product

In [ ]:
nutrition_df['product_name'].value_counts()

In [ ]:
spaghetti_df = nutrition_df[nutrition_df['product_name'] == 'Spaghetti']
print(spaghetti_df.shape)
spaghetti_df.head()

In [ ]:
for key, value in spaghetti_df.loc[979].items():
    if not pd.isna(value):
        print(key, "=", value)

In [ ]:
import httpx
def get_image_by_url(image_url: str, timeout_sec: int = 120) -> Image:
    with httpx.Client() as client:
        response = client.get(image_url, timeout=timeout_sec)
        return Image.open(BytesIO(response.content))

#### Front Image

In [ ]:
 get_image_by_url("https://images.openfoodfacts.org/images/products/000/000/003/2155/front_en.3.400.jpg")

#### Nutritional Info Image

In [ ]:
get_image_by_url("https://images.openfoodfacts.org/images/products/000/000/003/2155/nutrition_en.14.400.jpg")

### NutriScore 

* **Wiki** - https://en.wikipedia.org/wiki/Nutri-Score#Overview_of_algorithm
* **pypi** **package** - https://pypi.org/project/pyNutriScore/
  * helps to calculate both score and class for given nutritional fact data

In [ ]:
!pip install pyNutriScore

In [ ]:
from pyNutriScore import NutriScore

nutrifacts_data = {
        'energy': 1494.0,
        'fibers': 5.3571428571429,
        'fruit_percentage': 0,
        'proteins': 12.5,
        'saturated_fats': 0,
        'unsaturated_fats': 1.785,
        'sodium': 0,
        'sugar': 1.785,
        'carbohydrates': 75,
    }
food_type = 'solid'  # either 'solid' or 'beverage'

nutri_score, nutri_class = NutriScore().calculate(nutrifacts_data, food_type), NutriScore().calculate_class(nutrifacts_data, food_type)

print(nutri_score, nutri_class)

## Nutritional Facts Columns

In [ ]:
nutri_cols = [col for col in nutri_cols if col.endswith("_100g")]
len(nutri_cols), nutri_cols[:10]

## Check Completeness of Each Nutrient

In [ ]:
completeness = nutrition_df[nutri_cols].notna().sum().sort_values(ascending=False)

completeness_percent = completeness / len(nutrition_df) * 100

completeness_df = pd.DataFrame({
    "nutrient": completeness.index,
    "count": completeness.values,
    "percent": completeness_percent.values
})

# Top 30 nutrients by availability
top_df = completeness_df.head(30)

plt.figure(figsize=(10, 8))
sns.barplot(data=top_df, y="nutrient", x="percent", palette="crest")
plt.title("Top 30 Most Available Nutrients (% of Products)")
plt.xlabel("Completeness (%)")
plt.ylabel("Nutrient")
plt.show()

* Nutrients like energy_100g, fat_100g, carbohydrates_100g, sugars_100g, and proteins_100g have >90% coverage.

* Nutrients like calcium_100g, iron_100g, cholestrol_100g, trans-fat_100g and others have <10% coverage.

## Dominant Nutrients

In [ ]:
dominant = completeness_df[completeness_df["percent"] >= 20]
print(f"Dominant nutrients (≥20% completeness): {len(dominant)}")
dominant.head(10)

* These are the nutrients worth analyzing further for distribution and correlation.

# Data Distribution of Key Nutrients

In [ ]:
key_nutrients = dominant["nutrient"]

df_key = nutrition_df[key_nutrients]

# Clean up invalid values (negative or extreme outliers)
df_key = df_key[(df_key >= 0) & (df_key < df_key.quantile(0.99))]

df_key.plot(kind='box', figsize=(25, 15), logy=True)
plt.title("Boxplot of Major Nutrients per 100g (log scale)")
plt.ylabel("Value (g or kcal per 100g)")
plt.xticks(rotation=45) 
plt.show()


## Correlation Heatmap among key nutrients

In [ ]:
plt.figure(figsize=(8, 6))
corr = df_key.corr()
sns.heatmap(corr, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation Heatmap of Key Nutrients")
plt.show()

* energy-kj_100g and energy-kcal_100g highly correlated (~1.0).

* fat_100g correlates with energy,

* sugars_100g with carbohydrates_100g.

* proteins_100g has negative correlation with carbohydrates_100g and sugars_100g.

## Download Data for OCR Task 2

* For OCR Task, we will need to download products images data for all or sample nutritional food products.
* We also limit products nutritional info images to English language, otherwise will need multi-lingual OCR model
* Downloading images for all 0.8 Million food products would be time-consuming, so we will take the 10k sample products data in a way that:
  * maximizes nutrient coverage (so all 126 nutrients appear at least a few times)
  * ensures diversity (not just the common ones with full data like energy/fat/protein).
  * it helps an OCR model see diverse nutrient layouts (macros, vitamins, acids, etc.).

In [ ]:
nutrition_df['is_image_en_lang'] = nutrition_df["image_nutrition_url"].map(lambda url: 'nutrition_en' in url)
nutrition_df['is_image_en_lang'].value_counts()

In [ ]:
nutritions_eng_df = nutrition_df[nutrition_df['is_image_en_lang']]

completeness = nutritions_eng_df[nutri_cols].notna().sum().sort_values(ascending=False)

completeness_percent = completeness / len(nutrition_df) * 100

completeness_df = pd.DataFrame({
    "nutrient": completeness.index,
    "count": completeness.values,
    "percent": completeness_percent.values
})
completeness_df

In [ ]:
nutritions_eng_df.shape, completeness_df.shape

In [ ]:
SAMPLE_SIZE = 10_000 # 10k food products

df = nutritions_eng_df.drop_duplicates(subset='code').reset_index(drop=True)
comp = completeness_df.copy()

#Identify rare nutrients (bottom 20 or where < 0.1% coverage)
rare_nutrients = comp[comp["percent"] < 0.1]["nutrient"].tolist()

# Find rows where each rare nutrient is available
rare_rows = set()
for col in rare_nutrients:
    rare_rows.update(df[df[col].notna()].index.tolist())

rare_df = df.loc[list(rare_rows)]
print(f"Rows with rare nutrients: {len(rare_df)}")

# Compute row completeness across all nutrients
df["filled_count"] = df.notna().sum(axis=1)
df["filled_fraction"] = df["filled_count"] / df.shape[1]

# Sample remaining rows (with high completeness) to reach SAMPLE_SIZE total
remaining_needed = max(0, SAMPLE_SIZE - len(rare_df))
dense_df = df[df["filled_fraction"] >= 0.3]  # threshold adjustable

random_df = dense_df.sample(n=remaining_needed, random_state=42)
sample_df = pd.concat([rare_df, random_df]).drop_duplicates().reset_index(drop=True)
print("Sample Data shape", sample_df.shape)
sample_df.head()

In [ ]:
sample_completeness = sample_df[nutri_cols].notna().sum().sort_values(ascending=False)

sample_completeness_percent = completeness / len(nutrition_df) * 100

sample_completeness_df = pd.DataFrame({
    "nutrient": sample_completeness.index,
    "count": sample_completeness.values,
    "percent": sample_completeness_percent.values
})
print(sample_completeness_df.shape)
sample_completeness_df

In [ ]:
sample_df.to_csv("sample_products.csv", index=False)

In [ ]:
for key, value in sample_df.loc[0].items():
    if not pd.isna(value):
        print(key, "=", value)

## Data Download Using multi-processing

In [ ]:
from tenacity import retry, wait_fixed, stop_after_attempt

DATA_DIR = "products_images"

os.makedirs(DATA_DIR, exist_ok=True)

@retry(wait=wait_fixed(2), stop=stop_after_attempt(5))
def download_and_save(row: pd.Series):
    product_code = row['code']
    image_url = row['image_nutrition_url']

    if 'invalid' in image_url:
        image_url = image_url.replace('invalid', product_code)
    
    image_path = f"{DATA_DIR}/{product_code}.png"
    if not os.path.exists(image_path):
        try:
            image = get_image_by_url(image_url)
            image.save(image_path)
        except Exception as e:
            print("Failed to download image for url:", image_url)
            raise e

In [ ]:
from multiprocessing import Pool
from tqdm import tqdm

NUM_WORKERS = 12

with Pool(processes=NUM_WORKERS) as pool:
    list(
            tqdm(
                pool.imap(download_and_save, [row for _, row in sample_df[['code', 'image_nutrition_url']].iterrows()]),
                total=len(sample_df),
                dynamic_ncols=True,
            )
        )

In [ ]:
!ls